In [ ]:
# Views 3
from viewser.operations import fetch
from viewser import Queryset, Column
import views_runs
from views_partitioning import data_partitioner, legacy
from stepshift import views
from views_runs import storage, ModelMetadata
from views_runs.storage import store, retrieve, fetch_metadata
from views_forecasts.extensions import *

# For mapping 
import os
from views_mapper2.mapper2 import *
from views_mapper2.BBoxWriter import *
from views_mapper2.dictionary_writer import *
from views_mapper2.label_writer import *
from ingester3.extensions import *

In [ ]:
import geopandas as gpd
import sqlalchemy as sa
from ingester3.config import source_db_path

#Fetching geometry and matching to VIEWS country id's

engine = sa.create_engine(source_db_path)
gdf_ci_master = gpd.GeoDataFrame.from_postgis(
    "SELECT id as country_id, name, in_africa, in_me, geom FROM prod.country",
    engine,
    geom_col='geom'
)
gdf_ci_master = gdf_ci_master.to_crs(4326)

In [ ]:
import os
home = os.path.expanduser("~")
desktop = home+'/Desktop/' # (Toggle on to save to desktop) Change to your directory

In [ ]:
calib_run_id = 46 #For fatalities002 use 46 

In [ ]:
run = pd.DataFrame.forecasts.read_store(run=calib_run_id, name='cm_genetic_ensemble_f535')

In [ ]:
#Regional maps of surrogate models 
#Adjust region, textbox, title, column= and filename.
data = run.copy() # replace with one of the dataframes defined above: e.g., surrogate_CH, surrogate_POP etc.
gdf = gdf_ci_master.copy()

data = data.join(gdf.set_index("country_id"))
gdf = gpd.GeoDataFrame(data, geometry="geom")

In [ ]:
run # print to learn column names to plot

In [ ]:
#To see examples of what to do https://github.com/prio-data/Mapper2/blob/main/Mapper2_showcase.ipynb

region = 'ame' #options are 'ame', 'me' or 'globe'

textbox = 'Predicted log fatalities for state based violence' #Change the text in the textbox below the logo

test_map=Mapper2(
    width=10,   #dimension width
    height=10,  #dimension height
    frame_on=True,
    title="Predicted log fatalities in October 2026", #title ex: Predicted log fatalities, isolated contribution from infant mortality rate
    bbox=bbox_from_cid(f'{region}'), 
).add_layer(
    gdf=gdf.loc[536],  #what do you want to map, which month_id
    map_dictionary = dictionary_stand_1p_10k, #standard for cm maps is dictionary_stand_1p_10k
    cmap = 'rainbow', #rainbow is standard for cm maps, binary is greyscale 
    transparency = 1,
    background = None, 
    edgecolor="black",   #border color choice
    linewidth=0.5,    #line size choice
    column="step_combined"   # which column do you want to graph
).add_views_textbox(
    text=textbox)

test_map.save(desktop+'/prediction_map_rainbow', dpi=350) #change the filename
print('All done!')